In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
import tensorflow as tf
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [3]:
from keras.backend import set_session
set_session(sess)

Using TensorFlow backend.


In [4]:
import gym
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D

In [5]:
class PGAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.states = []
        self.gradients = []
        self.rewards = []
        self.probs = []
        self.model = self._build_model()
        self.model.summary()

    def _build_model(self):
        model = Sequential()
        model.add(Reshape((80, 80, 1), input_shape=(self.state_size,)))
        
        ############ large as DQN (fail in my implement)
#         model.add(Convolution2D(16,kernel_size=8,strides=1,activation='relu'))
#         model.add(Convolution2D(32,kernel_size=4,strides=1,activation='relu'))
#         model.add(Flatten())    
#         model.add(Dense(256, activation='relu'))
        ############
        
        ############ my
#         model.add(Convolution2D(16,kernel_size=8,strides=2,activation='relu'))
#         model.add(Convolution2D(32,kernel_size=4,strides=2,activation='relu'))
#         model.add(Flatten())        
        ############
        
        ############ original
        model.add(Convolution2D(32, 6, 6, subsample=(3, 3), border_mode='same',
                                activation='relu', init='he_uniform'))
        model.add(Flatten())
        model.add(Dense(64, activation='relu', init='he_uniform'))
        model.add(Dense(32, activation='relu', init='he_uniform'))
        ############
        model.add(Dense(self.action_size, activation='softmax'))
        opt = Adam(lr=self.learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer=opt)
        return model

    def remember(self, state, action, prob, reward):
        y = np.zeros([self.action_size])
        y[action] = 1
        self.gradients.append(np.array(y).astype('float32') - prob)
        self.states.append(state)
        self.rewards.append(reward)

    def act(self, state):
        state = state.reshape([1, state.shape[0]])
        aprob = self.model.predict(state, batch_size=1).flatten()
        self.probs.append(aprob)
        prob = aprob / np.sum(aprob)
        action = np.random.choice(self.action_size, 1, p=prob)[0]
        return action, prob

    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, rewards.size)):
            if rewards[t] != 0:
                running_add = 0
            running_add = running_add * self.gamma + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    def train(self):
        gradients = np.vstack(self.gradients)
        rewards = np.vstack(self.rewards)
        rewards = self.discount_rewards(rewards)
        rewards = rewards / np.std(rewards - np.mean(rewards))
        gradients *= rewards
        X = np.squeeze(np.vstack([self.states]))
        Y = self.probs + self.learning_rate * np.squeeze(np.vstack([gradients]))
        self.model.train_on_batch(X, Y)
        self.states, self.probs, self.gradients, self.rewards = [], [], [], []

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
def preprocess(I):
    I = I[35:195]
    I = I[::2, ::2, 0]
    I[I == 144] = 0
    I[I == 109] = 0
    I[I != 0] = 1
    return I.astype(np.float).ravel()

In [ ]:
env = gym.make("Pong-v0")
state = env.reset()
prev_x = None
score = 0
episode = 0

state_size = 80 * 80
action_size = env.action_space.n
agent = PGAgent(state_size, action_size)
# agent.load('pong.h5')
start_time = time.time()
now = start_time
while True:
#     env.render()

    cur_x = preprocess(state)
    x = cur_x - prev_x if prev_x is not None else np.zeros(state_size)
    prev_x = cur_x

    action, prob = agent.act(x)
    state, reward, done, info = env.step(action)
    score += reward
    agent.remember(x, action, prob, reward)

    if done:
        episode += 1
        agent.train()
        print('Episode: %d - Score: %f. use %.2fs, totally %.2fs' % (episode, score, time.time()-now, time.time()-start_time))
        now = time.time()
        score = 0
        state = env.reset()
        prev_x = None
        if episode > 1 and episode % 10 == 0:
            agent.save('pong.h5')

[2018-01-08 20:31:12,507] Making new env: Pong-v0
/home/jiancheng/install/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (6, 6), padding="same", kernel_initializer="he_uniform", strides=(3, 3), activation="relu")`
/home/jiancheng/install/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, kernel_initializer="he_uniform", activation="relu")`
/home/jiancheng/install/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, kernel_initializer="he_uniform", activation="relu")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 80, 80, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 32)        1184      
_________________________________________________________________
flatten_1 (Flatten)          (None, 23328)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1493056   
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 1,496,518
Trainable params: 1,496,518
Non-trainable params: 0
_________________________________________________________________
